In [1]:
from maze_common import *
import time
import random as rnd
import numpy as np
import copy
import matplotlib.pyplot as plt

Generate a maze.

In [18]:
maze = Maze(60, .30)
solvable = maze.isSolvable()
print("Is solvable?", solvable)
print("Obstacles?", maze.obstacles.shape[0])

If the maze is solvable, find the shortest path *without* a heuristic function (i.e. uniform cost search).

In [17]:
if (solvable):
    shortestPath, expandedCells = shortestPathSearch(maze)
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Define a heuristic that employs the Manhattan distance formula.

In [4]:
def manhattanDistance(cell, maze, visited):
    cellRow, cellCol = cell.coords
    return ((maze.dim - 1) - cellRow) + ((maze.dim - 1) - cellCol)

Test the Manhattan distance heuristic, and ensure the shortest path is valid and has length less than or equal to that of the original shortest path.

In [16]:
if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = manhattanDistance)
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Define a function for thinning a maze. Given the maze and a fraction of obstacles to remove, create a copy of the maze that has a reduced number of obstacles.

In [6]:
def thinMaze(maze, fractionRemove):
    thinMaze = copy.deepcopy(maze)
    # Choose how many obstacles to remove. This value is rounded to the nearest integer.
    numRemove = int(round(fractionRemove * thinMaze.obstacles.shape[0]))
    for i in range(0, numRemove):
        # Choose a random obstacle to remove by choosing a random index in the thinned maze's obstacle array.
        indexRemove = int(rnd.random() * thinMaze.obstacles.shape[0])
        obstacleX, obstacleY = thinMaze.obstacles[indexRemove]
        # Remove the chosen obstacle from the thinned maze's obstacle array.
        thinMaze.obstacles = np.delete(thinMaze.obstacles, indexRemove, axis=0)
        # Update the thinned maze's board to open up the previously blocked position.
        thinMaze.board[obstacleX, obstacleY] = Cell.OPEN
    return thinMaze
    
thinnedMaze = thinMaze(maze, .5)
print("Original obstacles: ", maze.obstacles.shape[0])
print("Thinned obstacles: ", thinnedMaze.obstacles.shape[0])

Original obstacles:  1115
Thinned obstacles:  557


Define a heuristic that finds the shortest path from the given coordinates to the goal in the thinned maze. This heuristic will simply use a uniform cost path search in the thinned maze from the given coordinates to the goal, and it has access to a dictionary of already-discovered heuristics that prevents redoing the heuristic for cells that have already been passed to this function. This `visited` dictionary is valid because the heuristic score should never change for a given cell, since the maze utilized is static.

In [7]:
def thinnedMazeShortestPathLength(cell, maze, visited):
    cellRow, cellCol = cell.coords
    shortestPathLength = 0
    if ((cellRow, cellCol)) not in visited:
        shortestPath, ignoredExpandedCellsBySearch = shortestPathSearch(thinnedMaze, startCoords = (cellRow, cellCol))
        shortestPathLength = len(shortestPath)       
        for i in range(0, shortestPathLength):
            row, col = shortestPath[i]
            if ((row, col)) in visited: 
                break
            else:
                visited[(row, col)] = shortestPathLength - i - 1
    else:
        shortestPathLength = visited[(cellRow, cellCol)]
    return shortestPathLength

Test the thinned maze heuristic, and ensure the shortest path is valid and has length less than or equal to that of the original shortest path.

In [13]:
if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = thinnedMazeShortestPathLength)
    print("Time:", time.time() - startTime, "seconds")
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Define a new function for finding neighboring cells. This function builds upon the `findNeighboringCoords` function defined in `maze_common.py`. However, we now introduce to the agent the option of moving diagonally towards the bottom right. This bottom right movement would be preferred to moving either to the bottom neighbor or moving to the right neighbor, so both of these neighbors are no longer considered if the bottom right neighbor is open. 

In [9]:
def findNeighboringOpenCoordsIncludingDiagonals(coords, maze):
    neighbors = findNeighboringOpenCoords(coords, maze)
    cellRow, cellCol = coords
    row, col = (cellRow + 1, cellCol + 1)
    if (row < maze.dim and row >= 0 and col < maze.dim and col >= 0 and maze.board[row,col] == Cell.OPEN):
        neighbors.append((row, col))
        # Try removing the bottom neighbor, if it is present.
        try:
            neighbors.remove((cellRow + 1, cellCol))
        except ValueError: 
            ""
        # Try removing the right neighbor, if it is present.
        try:
            neighbors.remove((cellRow, cellCol + 1))
        except ValueError:
            ""
    
    return neighbors

Define a heuristic that finds the shortest path from the given coordinates to the goal in the original maze with the "enhanced" `findNeighbors` function that takes into account the bottom-right diagonal neighbor as well. This heuristic will simply use a uniform cost path search in the maze from the given coordinates to the goal, and it has access to a dictionary of already-discovered heuristics that prevents redoing the heuristic for cells that have already been passed to this function. This `visited` dictionary is valid because the heuristic score should never change for a given cell, since the maze utilized is static.

In [10]:
def diagonalTravelShortestPathLength(cell, maze, visited):
    cellRow, cellCol = cell.coords 
    shortestPathLength = 0
    if ((cellRow, cellCol)) not in visited:
        shortestPath, ignoredExpandedCellsBySearch = shortestPathSearch(maze, startCoords = (cellRow, cellCol), findNeighborsFunction = findNeighboringOpenCoordsIncludingDiagonals)
        shortestPathLength = len(shortestPath)
        for i in range(0, shortestPathLength):
            row, col = shortestPath[i]
            if ((row, col)) in visited: 
                break
            else:
                visited[(row, col)] = shortestPathLength - i - 1
    else:
        shortestPathLength = visited[(cellRow, cellCol)]
    return shortestPathLength

Test the diagonal travel heuristic, and ensure the shortest path is valid and has length less than or equal to that of the original shortest path.

In [12]:
if (solvable):
    startTime = time.time()
    shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = diagonalTravelShortestPathLength)
    print("Time:", time.time() - startTime, "seconds")
    print("Expanded cells:", expandedCells)
    print("Shortest path length:", len(shortestPath), "\nShortest path:", shortestPath)
    boardWithPath = copy.deepcopy(maze.board)
    for coords in shortestPath:
        row, col = coords
        if (maze.board[row, col] == -1):
            print("Error")

Perform testing for all 3 previously defined heuristics. Warning: this takes an incredibly long time to run.

In [1]:
p = 0.3
dim = 50

iterationsPerRho = 100
rhos = np.arange(0.05, 1, 0.1)

manhattanDistanceExpandedCells = np.zeros([rhos.size, iterationsPerRho])
manhattanDistanceTimeCosts = np.zeros([rhos.size, iterationsPerRho])
thinnedMazeExpandedCells = np.zeros([rhos.size, iterationsPerRho])
thinnedMazeTimeCosts = np.zeros([rhos.size, iterationsPerRho])
diagonalTravelExpandedCells = np.zeros([rhos.size, iterationsPerRho])
diagonalTravelTimeCosts = np.zeros([rhos.size, iterationsPerRho])

testStartTime = time.time()

for i in range(0, rhos.size):
    rho = rhos[i]
    breakset = False
    for j in range(0, iterationsPerRho):
        print("Testing rho =", rho, "Iteration:", j)
        maze = Maze(dim, p)
        while not maze.isSolvable():
            maze = Maze(dim, p)
        openCells = maze.dim**2 - maze.obstacles.shape[0]
        
        startTime = time.time()
        shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = manhattanDistance)
        endTime = time.time() - startTime
        manhattanDistanceExpandedCells[i, j] = expandedCells / openCells
        manhattanDistanceTimeCosts[i, j] = endTime
        print("\tManhattan Distance Time:", endTime, "seconds")
        
        thinnedMaze = thinMaze(maze, rho)
        startTime = time.time()
        shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = thinnedMazeShortestPathLength)
        endTime = time.time() - startTime
        thinnedMazeExpandedCells[i, j] = expandedCells / openCells
        thinnedMazeTimeCosts[i, j] = endTime
        print("\tMaze Thinning Time:", endTime, "seconds")
        
        startTime = time.time()
        shortestPath, expandedCells = shortestPathSearch(maze, heuristicFunction = diagonalTravelShortestPathLength)
        endTime = time.time() - startTime
        diagonalTravelExpandedCells[i, j] = expandedCells / openCells
        diagonalTravelTimeCosts[i, j] = endTime
        print("\tDiagonal Travel Time:", endTime, "seconds")
        
manhattanDistanceAvgExpandedCells = np.mean(manhattanDistanceExpandedCells)
manhattanDistanceAvgTimeCost = np.mean(manhattanDistanceTimeCosts)
diagonalTravelAvgExpandedCells = np.mean(diagonalTravelExpandedCells)
diagonalTravelAvgTimeCost = np.mean(diagonalTravelTimeCosts)

thinnedMazeAvgExpandedCells = np.zeros([rhos.size])
thinnedMazeAvgTimeCosts = np.zeros([rhos.size])
for i in range(0, rhos.size):
    thinnedMazeAvgExpandedCells[i] = np.mean(thinnedMazeExpandedCells[i])
    thinnedMazeAvgTimeCosts[i] = np.mean(thinnedMazeTimeCosts[i])

print("**Testing took", time.time() - testStartTime, "seconds to complete.**")
print("Manhattan Distance Avg Expanded Cells:", manhattanDistanceAvgExpandedCells)
print("Manhattan Distance Avg Time Cost:", manhattanDistanceAvgTimeCost)
print("Thinned Maze Avg Expanded Cells:", thinnedMazeAvgExpandedCells)
print("Thinned Maze Avg Time Costs:", thinnedMazeAvgTimeCosts)
print("Diagonal Travel Avg Expanded Cells:", diagonalTravelAvgExpandedCells)
print("Diagonal Travel Avg Time Cost:", diagonalTravelAvgTimeCost)

Generate the plots for the expanded cells (as a fraction of open cells) and for the runtimes (in seconds) for each heuristic function.

In [ ]:
#Expanded Cells Graph
plt.figure(figsize=(12,8))
plt.hlines(manhattanDistanceAvgExpandedCells, np.amin(rhos), np.amax(rhos), color="red", label="Manhattan Distance")
plt.plot(rhos, thinnedMazeAvgExpandedCells, label="Thinned Maze")
plt.hlines(diagonalTravelAvgExpandedCells, np.amin(rhos), np.amax(rhos), color="green", label="Diagonal Travel")
plt.xlabel("Rho Values")
plt.ylabel("Expanded Cells (as a fraction of open cells)")
plt.title("Expanded Cells In Each Heuristic")
plt.legend(loc="best")
plt.show()

#Average Time Cost Graph
plt.figure(figsize=(12,8))
plt.hlines(manhattanDistanceAvgTimeCost, np.amin(rhos), np.amax(rhos), color="red", label="Manhattan Distance")
plt.plot(rhos, thinnedMazeAvgTimeCosts, label="Thinned Maze")
plt.hlines(diagonalTravelAvgTimeCost, np.amin(rhos), np.amax(rhos), color="green", label="Diagonal Travel")
plt.xlabel("Rho Values")
plt.ylabel("Time Cost (in seconds)")
plt.title("Average Time Cost of Each Heuristic")
plt.legend(loc="best")
plt.show()